In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#path_post_enc = '/content/drive/Othercomputers/My Laptop/post_enc_frames'
path_post_enc = '/content/drive/MyDrive/post_enc_frames2'
path_pre_enc = '/content/drive/Othercomputers/My Laptop/pre_enc_frames'
batch_size = 8

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras.layers import (Dense, Concatenate, Add, ReLU, Input, GlobalAveragePooling2D, Activation,
                                     Conv1D, Multiply, Permute, Conv3D)
from tensorflow.keras import optimizers
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from matplotlib import pyplot as plt
from keras.callbacks import CSVLogger
from collections import deque
import gc
import copy
import random

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

root = os.getcwd()

In [5]:
folders_post = sorted(os.listdir(path_post_enc))
folders_pre = sorted(os.listdir(path_pre_enc))

In [6]:
def get_folder_indices(start_index, batch_size, frame_num=128):
    folder_idx = start_index
    indices = deque()

    while len(indices) != batch_size:
        os.chdir(os.path.join(path_post_enc, folders_post[folder_idx]))
        files = os.listdir(os.getcwd())


        if len(files) >= frame_num:
            indices.append(folder_idx)
        folder_idx += 1

    os.chdir(root)

    return indices, folder_idx

def data_generator(start_index, batch_size=batch_size):
    indices, start_index = get_folder_indices(start_index=start_index, batch_size=batch_size, frame_num=128)
    indices_copy = copy.deepcopy(indices)

    X = np.empty(shape=(batch_size, 128, 256, 256, 1), dtype="float32")
    y = np.empty(shape=(batch_size, 128, 256, 256, 1), dtype="float32")

    # post enc
    m = 0
    while len(indices) > 0:
        os.chdir(os.path.join(path_post_enc, folders_post[indices.popleft()]))
        files = os.listdir(os.getcwd())
        for i, file in enumerate(files):
            if i < 128:
                with open(file, 'rb') as yuv_file:
                    X[m][i] = np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1))
        m += 1

    # pre enc
    m = 0
    while len(indices_copy) > 0:
        os.chdir(os.path.join(path_pre_enc, folders_pre[indices_copy.popleft()]))
        files = os.listdir(os.getcwd())
        for i, file in enumerate(files):
            if i < 128:
                with open(file, 'rb') as yuv_file:
                    y[m][i] = np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1))
        m += 1

    os.chdir(root)

    return X / 255.0, y / 255.0, start_index


In [ ]:

'''
for y in range(0, 8*50, 8):
  X_train, y_train, st_ix = data_generator(y, 8)

  for i in range(8):
    print(np.average(X_train[i]))
    print(np.average(y_train[i]))
    print(" ")
'''



'\nfor y in range(0, 8*50, 8):\n  X_train, y_train, st_ix = data_generator(y, 8)\n\n  for i in range(8):\n    print(np.average(X_train[i]))\n    print(np.average(y_train[i]))\n    print(" ")\n'

In [7]:
def dc_block(input):
    x = Conv3D(filters=4, kernel_size=3, strides=1, padding='same')(input)

    x1 = Dense(6, activation='relu')(x)
    x1 = Conv3D(filters=6, kernel_size=3, strides=1,
                padding='same', activation='relu')(x1)

    x2 = Dense(8, activation='relu')(x1)
    x2 = Dense(8, activation='relu')(x2)
    x2 = Conv3D(filters=8, kernel_size=3, strides=1,
                padding='same', activation='relu')(x2)

    x3 = Dense(10, activation='relu')(x2)
    x3 = Dense(10, activation='relu')(x3)
    x3 = Dense(10, activation='relu')(x3)
    x3 = Conv3D(filters=10, kernel_size=3, strides=1,
                padding='same', activation='relu')(x3)

    x4 = Dense(12, activation='relu')(x3)
    x4 = Dense(12, activation='relu')(x4)
    x4 = Dense(12, activation='relu')(x4)
    x4 = Dense(12, activation='relu')(x4)
    x4 = Conv3D(filters=12, kernel_size=3, strides=1,
                padding='same', activation='relu')(x4)

    x5 = Dense(12, activation='relu')(x4)
    x5 = Concatenate()([Dense(4)(input), x5])
    x5 = Conv3D(filters=8, kernel_size=3, strides=1,
                padding='same', activation='relu')(x5)

    x6 = Add()([x5, input])
    out = Conv3D(filters=8, kernel_size=3, strides=1,
                 padding='same', activation='relu')(x6)

    return out


def dcr_net(input_shape=(None, None, None, 1), filters=8,
            name='DCRN_model'):  # ORIGINAL 64 filters!!!
    lr_image = Input(shape=input_shape, name='input')
    x_start = Conv3D(filters, kernel_size=3, strides=1,
                     padding='same')(lr_image)
    x_start = ReLU()(x_start)
    x = dc_block(x_start)
    x1 = dc_block(x)
    x2 = dc_block(x1)
    x3 = Concatenate()([Dense(8)(x_start), x, x1, x2])
    x4 = Conv3D(filters, kernel_size=3, strides=1, padding='same')(x3)
    x4 = ReLU()(x4)
    # x5 = cab(x4)
    x5 = Add()([x4, x_start])
    out = Conv3D(1, kernel_size=3, strides=1, padding='same',
                 dtype='float64')(x5)
    return tf.keras.Model(inputs=lr_image, outputs=out, name=name)


def cab(input):
    x = GlobalAveragePooling2D()(input)
    x = Permute((2, 1))(x)  # Act on channel axis
    x = Conv1D()(x)
    x = ReLU()(x)
    x = Conv1D()(x)
    x = Activation('sigma')(x)
    x = Permute((2, 1))(x)  # Restore original channel axis
    out = Multiply()([x, input])
    return out

In [8]:
Model = dcr_net()

csv_logger = CSVLogger('/content/drive/MyDrive/log.csv', append=True, separator=';')

checkpoint_filepath = '/content/drive/MyDrive/3Dmodel2_checkpoint_lion_20_05_24.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='max',
    save_best_only=False,
    save_weights_only=False,
    save_freq="epoch")

In [ ]:
Model = tf.keras.models.load_model(checkpoint_filepath) # ÇOK DİKKAT ET BU KAPALI BUNUN KAPALI OLMASI GERKEN DURUMLAR OLABİLİR


In [ ]:
optimizer = optimizers.Adam(learning_rate=1e-4)
Model.compile(optimizer=optimizer, loss='mse', metrics=["mse", "mae"])

In [ ]:
# epochs = 460 (0.8 test size)
st_ix = 0
epochs = 11
file_path = '/content/drive/MyDrive/devam_indisi.txt'
kac_epoch = '/content/drive/MyDrive/d_kac_epoch.txt'

for i in range(epochs):
    while True:
        with open(file_path, 'r') as file:
            st_ix = int(file.read())

        if st_ix >= 456:
            with open(file_path, 'w') as file:
                file.write("0")
            with open(kac_epoch, 'r') as file:
                a = file.read()
                a = int(a)
            with open(kac_epoch, 'w') as file:
                a += 1
                file.write(f"{a}")
            break

        X_train, y_train, st_ix = data_generator(st_ix, 8)

        rand1 = random.randint(0, 128)
        Model.fit(
            x=X_train[:, :, rand1:rand1+128, rand1:rand1+128],
            y=y_train[:, :, rand1:rand1+128, rand1:rand1+128],
            epochs=1,
            batch_size=1,
            shuffle=True,
            callbacks=[model_checkpoint_callback]
        )


        with open(file_path, 'w') as file:
            file.write(f"{st_ix}")

        gc.collect()


8/8 [==============================] - ETA: 0s - loss: 6.8645e-04 - mse: 6.8645e-04 - mae: 0.0153

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 18s 2s/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0241


In [ ]:
gc.collect()
with open('/content/drive/MyDrive/devam_indisi.txt', 'r') as file:
    st_ix = int(file.read())


In [ ]:
pred_loss_list = []
x_loss_list = []
pred_psnr_list = []
x_psnr_list = []
batch_size = 8
coef=5

predictions = np.empty(shape=(batch_size, 128, 256, 256, 1), dtype="float32")
for k in range(464, 565, 8):
  X_val, y_val, _ = data_generator(start_index=k, batch_size=batch_size)
  for i in range(2):
    for j in range(2):
      predictions[:, :, i*128:(i+1)*128, j*128:(j+1)*128] = Model.predict(X_val[:, :, i*128:(i+1)*128, j*128:(j+1)*128])
  pred_loss = np.sum(np.square(y_val - predictions))/(batch_size*128)
  x_loss = np.sum(np.square(X_val - y_val))/(batch_size*128)
  pred_psnr = 20*np.log10(255/np.sqrt(pred_loss))
  x_psnr = 20*np.log10(255/np.sqrt(x_loss))

  print(f"pred loss: {pred_loss} PSNR:{pred_psnr}")
  print(f"x_loss: {x_loss} PSNR:{x_psnr}")
  pred_loss_list.append(pred_loss)
  x_loss_list.append(x_loss)
  pred_psnr_list.append(pred_psnr)
  x_psnr_list.append(x_psnr)
  '''
  for i in range(8):
    max_psnr_fark = 0
    max_j = 0
    print(f"index: {i}")

    for j in range(127):
      pred_loss = np.sum(np.square(y_val[i][j] - predictions[i][j]))
      x_loss = np.sum(np.square(X_val[i][j] - y_val[i][j]))
      PSNR_pred = 20*np.log10(255.0/np.sqrt(pred_loss))
      PSNR_x = 20*np.log10(255.0/np.sqrt(x_loss))
      max_psnr_fark = PSNR_pred - PSNR_x
      max_j = j
      print(f"{j} {PSNR_pred} {max_psnr_fark}")
    '''
    #print(f"{i}: max psnr fark: {max_psnr_fark} index: {j}")


  #pred_loss_list.append(np.sum(np.square(y_val - predictions))/(batch_size*128))
  #x_loss_list.append(np.sum(np.square(X_val - y_val))/(batch_size*128))


#################### BURANIN AŞAĞISI ÖNEMSİZ ######################

print(f"Pred MSE average {np.average(np.array(pred_loss_list))}")
print(f"X MSE average {np.average(np.array(x_loss_list))}")
print(f"Pred PSNR average {np.average(np.array(pred_psnr_list))}")
print(f"X PSNR average {np.average(np.array(x_psnr_list))}")



1/1 [==============================] - 68s 68s/step
pred loss: 35.70656967163086 PSNR:32.60332231320346
x_loss: 72.71331787109375 PSNR:29.514663991392883
1/1 [==============================] - 68s 68s/step
pred loss: 38.77634811401367 PSNR:32.245134552642924
x_loss: 73.9036636352539 PSNR:29.444143926065625
1/1 [==============================] - 68s 68s/step
pred loss: 28.05306053161621 PSNR:33.65100112049883
x_loss: 54.29924774169922 PSNR:30.78286547925326
1/1 [==============================] - 68s 68s/step
pred loss: 36.405399322509766 PSNR:32.51914561777194
x_loss: 69.55741119384766 PSNR:29.707369509235026
1/1 [==============================] - 68s 68s/step
pred loss: 25.884069442749023 PSNR:34.00047804574708
x_loss: 51.66008758544922 PSNR:30.999252227192482
1/1 [==============================] - 68s 68s/step
pred loss: 28.311603546142578 PSNR:33.61115892720796
x_loss: 54.400543212890625 PSNR:30.774771245303363
1/1 [==============================] - 68s 68s/step
pred loss: 50.1765899

In [ ]:
cv2.imwrite(f"/content/drive/MyDrive/gorseller/pred.png", predictions[1,72] * 255.0)
cv2.imwrite(f"/content/drive/MyDrive/gorseller/xval.png", X_val[1,72] * 255.0)
cv2.imwrite(f"/content/drive/MyDrive/gorseller/yval.png", y_val[1,72] * 255.0)


True

In [ ]:
# 472, 520, 8
for i in range(48):
  for j in range(128):
    max = 0
    pred_loss = np.sum(np.square(y_val - predictions))/(batch_size*128)
    x_loss = np.sum(np.square(X_val - y_val))/(batch_size*128)



In [ ]:
for i in range(6):
  print(f"X loss: {x_loss_list[i]} Prediction loss: {pred_loss_list[i]}")


In [ ]:
pred_loss = 0
x_loss = 0

predictions = np.empty(shape=(batch_size, 128, 256, 256, 1), dtype="float32")

for i in range(2):
  for j in range(2):
    for k in range(0, batch_size, 10):
      predictions[k:k+10, :, i*128:(i+1)*128, j*128:(j+1)*128] = Model.predict(X_val[k:k+10, :, i*128:(i+1)*128, j*128:(j+1)*128])


predictions[predictions < 0.0] = 0.0
predictions[predictions > 1.0] = 1.0

1/1 [==============================] - 5s 5s/step


In [ ]:
pred_loss = np.sum(np.square(y_val - predictions))/(batch_size*128)
x_loss = np.sum(np.square(X_val - y_val))/(batch_size*128)

In [ ]:
print(f"pred loss: {pred_loss}")
print(f"x_loss: {x_loss}")

pred loss: 50.134151458740234
x_loss: 86.61634826660156


In [ ]:
x_loss = np.sum(np.square(X_val - y_val))/(8*128)
print(f"x_loss: {x_loss}")

In [ ]:
idx=52
cv2.imwrite(f"/content/drive/MyDrive/gorseller/pred.png", predictions[3,idx] * 255.0)
cv2.imwrite(f"/content/drive/MyDrive/gorseller/x.png", X_val[3,idx] * 255.0)
cv2.imwrite(f"/content/drive/MyDrive/gorseller/y.png", y_val[3,idx] * 255.0)


In [ ]:
cv2.imwrite(f"/content/drive/MyDrive/gorseller/p2.png", X_val[3,50] * 255.0)
cv2.imwrite(f"/content/drive/MyDrive/gorseller/p3.png", y_val[3,50] * 255.0)

